In [1]:
import datetime
import pickle

import pandas as pd
from river import metrics, optim, reco

In [2]:
with open("../data/model.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
df = pd.read_pickle("../data/trainset.pkl")
anime_list = df[["Name", "Japanese_name", "anime_id", "Score"]].drop_duplicates()

In [4]:
now = datetime.datetime.now()

In [5]:
user_id = now.strftime("%Y%m%d%H%M%S")
user_id = int(user_id)
print(user_id)

20230622192317


In [6]:
test_data = [
    "Angel Beats!",
    "Mahoutsukai Precure!",
    "Ookami to Koushinryou",
    "Shinsekai yori",
    "Shugo Chara!",
    "Uma Musume: Pretty Derby (TV)",
    "Fate/Zero",
    "Uchuu Senkan Yamato 2199",
]

In [7]:
test_df = anime_list[anime_list["Name"].isin(test_data)]
test_df["user_id"] = user_id
test_df["rating"] = 10

/tmp/ipykernel_447174/2289128984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["user_id"] = user_id
/tmp/ipykernel_447174/2289128984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["rating"] = 10


In [8]:
testset = test_df[["user_id", "anime_id", "rating"]].to_dict(orient="records")

In [9]:
metric = metrics.MAE() + metrics.RMSE()

In [10]:
# Update model
for data in testset:
    y_pred = model.predict_one(user=data["user_id"], item=data["anime_id"])
    metric = metric.update(data["rating"], y_pred)
    model = model.learn_one(
        user=data["user_id"], item=data["anime_id"], y=data["rating"]
    )

In [11]:
metric

MAE: 1.790617, RMSE: 1.822808

In [12]:
with open("../data/retrain_model.pkl", "wb") as f:
    pickle.dump(model, f)